In [2]:
# import library files
import json
from datetime import datetime

# read the product list
fd=open("product_lists.json",'r')
data1 = fd.read()
fd.close()

# convert string into dictionary
data = json.loads(data1)

# read the Transaction list
fd=open("txn_list.json",'r')
data2 = fd.read()
fd.close()

# convert string into dictionary
transaction_list = json.loads(data2)
#---------------------------------------------------------------FUNCTIONS---------------------------------------------------------
# Checking product id in database/list
def checkid(data,id):
  for i in data:
    if str(i)==str(id):
      return 1
  return 0

# Show the product list in table format
def list_prod(data):
  print("\nProducts List:")
  print("---------------------------------------------------------------------------")
  print("P_Id","Name\t\t","Price\t","Quantity\t","Expiry_date\t","Category")
  print("---------------------------------------------------------------------------")
  for i in data:
    print(int(i),data[i]['name'],"\t",data[i]['price'],"\t",data[i]['quantity'],"\t\t",data[i]['Expiry date'],"\t",data[i]['category'])
  print("---------------------------------------------------------------------------\n")

# Show the Transaction list in table format
def list_txn(tl):
  print("\nTransaction List:")
  print("---------------------------------------------------------------------------------------")
  print("Txn_Id\t\t\t","Item_count\t","Amount\t","Date & Time\t\t","Customer_Name")
  print("---------------------------------------------------------------------------------------")
  for i in tl:
    print(int(i),'\t',tl[i]['item_count'],"\t\t",tl[i]['amount'],"\t",tl[i]['time'],"\t",tl[i]['c_name'])
  print("---------------------------------------------------------------------------------------\n")

#--------------------------------------------------------------------------------------------------------------------------------------

print("You want to buy or add a products:")
print("1.Buy 2.Add/Remove")
mo=int(input("Enter 1 or 2: "))

# buy a products
if (mo==1):
  
  # show the product list in table format (calling the function)
  list_prod(data)

  # customer name
  cname=input("Enter your name: ")
  
  # id=[] list is used for store multiple products user wants to buy and also quantity
  id=[]
  qty=[]
  n1=int(input("How many products you want to buy: "))

  # for loop used for store product ids and qty
  for j in range(n1):
    print("\nProduct No:",j+1)
    id.append(input("Enter the product id: "))
    qty.append(input("Enter the quantity: "))
  
  # intilize sum =0 , it is used store the total bill amount
  sum=0
  
  # Retrive the system time or local time
  now = datetime.now()
  # dd/mm/YY H:M:S
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
 
  # txn_id is contain unique number generated by using system date and time combination. ex-> year+month+day+00+hour+minutes+secounds
  txn_id = now.strftime("%Y%m%d"+"00"+"%H%M%S")

  # Show the receipt/bill in table format
  print("\n----------------------Receipt-------------------------")
  print("Customer_Name:",cname)
  print("Txn_No:",txn_id,'\t',dt_string)
  print("------------------------------------------------------")
  print("P_Id","Name\t\t","Price\t","Quantity\t","Total")
  print("------------------------------------------------------")
  
  # get the product id from the list (already customer give the p_id) 
  for i in range(len(id)):
    # check the product availability
    if checkid(data,id[i]) == 0:
      print("\nProduct is not available\n")
      sys.exit(0)
    # Check the available qty from that Product id 
    if int(qty[i]) > data[id[i]]["quantity"]:
      print("\nThe availabe quantity is",data[id[i]]['quantity'],"\n")
      sys.exit(0)

    # calculate the bill and print the each product
    for j in data:
      if str(j)==id[i]:
        print(int(j),data[j]['name'],"\t",data[j]['price'],"\t",qty[i],"\t\t",int(qty[i])*data[j]['price'])
        sum=sum + int(qty[i])*data[j]['price']
        data[j]["quantity"] = data[j]["quantity"] - int(qty[i])
  print("------------------------------------------------------")
  print("Item Count:\t\t\t\t\t",n1)
  print("Bill Amount:\t\t\t\t\t", sum)
  print("------------------------------------------------------")

  # store/update the customer detail in the txn dictionary
  transaction_list[txn_id]={'time':dt_string,'amount': sum,'item_count': n1,'c_name':cname}

  # convert products dictionary into string
  data1 = json.dumps(data)
  # write/update the file
  fd=open("product_lists.json",'w')
  fd.write(data1)
  fd.close()

  # convert txn dictionary into string
  tx= json.dumps(transaction_list)
  # write/update the file
  fd=open("txn_list.json",'w')
  fd.write(tx)
  fd.close()

# add or remove or view the transaction details
elif mo==2:
  print("If you want to add a products or remove the products:")
  
  # view the products available in the store
  list_prod(data)

  print("1.Add 2.Remove 3.Show txn list")
  mo=int(input("Enter 1 or 2 or 3: "))
  # add the products
  if (mo==1):
    n= input("how many products to add: ")

    for i in range(int(n)):
      
      prod_id = input("Enter the product id: ")
      # if the product is exist
      if (prod_id in data.keys()):
        print("Product id:",prod_id, "is exist\n")
        print("The availabe quantity is:",data[prod_id]['quantity'])
        q=int(input("Enter the quantity to add: "))
        data[prod_id]['quantity']=int(data[prod_id]['quantity'])+q
        print("The quantity added sucessfully. Now available quantity is:",data[prod_id]['quantity'])
      
      # product is doesn't exist to add new product
      else:
        print("Product id:",prod_id, "doesn't exist. add new product\n")
        prod_name = input("Enter the product name: ")
        prod_price = int(input("Enter the product price: "))
        prod_qty = int(input("Enter the product quantity: "))
        prod_exd = input("Enter the product expiry date: ")
        prod_categ = input("Enter the product category: ")
        # update the products
        data[prod_id] = {"name":prod_name,"price":prod_price,"quantity":prod_qty,"Expiry date":prod_exd,"category":prod_categ}
        print("Product id:",prod_id, "is added sucessfully\n")
    
    # store the products dict into json file
    data1 = json.dumps(data)
    fd=open("product_lists.json",'w')
    fd.write(data1)
    fd.close()

  # remove the products
  elif mo==2:
    n= input("how many products to remove: ")

    for i in range(int(n)):
      
      prod_id = input("Enter the product id: ")
      # try and except used for the entered p_id is available execute "try" otherwise go to "except"
      try:
        del data[prod_id]
        print("Product id:",prod_id,"is removed sucessfully\n")
      except:
        print("Product id:",prod_id,"is Not available in the list\n")

    # update the product list
    data1 = json.dumps(data)
    fd=open("product_lists.json",'w')
    fd.write(data1)
    fd.close()

  # view the txn list
  elif mo==3:
    list_txn(transaction_list)
  # if entered input is except (1,2,3)------->add,remove,show txn
  else:
    print("Entered input is wroung")
# if entered input is except (1,2)------->buy,add/remove
else:
  print("\nEntered input was wroung\n")

You want to buy or add a products:
1.Buy 2.Add/Remove
Enter 1 or 2: 1

Products List:
---------------------------------------------------------------------------
P_Id Name		 Price	 Quantity	 Expiry_date	 Category
---------------------------------------------------------------------------
1001 Good day      	 16 	 50 		 04/04/2022 	 Biscuits
1002 Parle monaco  	 99 	 29 		 21/10/2021 	 Biscuits
1003 Nutrichoice   	 22 	 19 		 10/04/2022 	 Biscuits
1004 Mom's magic   	 90 	 30 		 15/11/2021 	 Biscuits
1005 Milk bikis    	 51 	 9 		 10/11/2021 	 Biscuits
1006 bourbon       	 22 	 19 		 01/12/2021 	 Biscuits
1007 50-50         	 63 	 9 		 11/09/2021 	 Biscuits
1008 Dark fantasy  	 66 	 14 		 11/09/2021 	 Biscuits
1009 Parle-G       	 68 	 20 		 11/01/2022 	 Biscuits
1010 Krack jack    	 54 	 14 		 01/01/2022 	 Biscuits
1011 Little hearts 	 19 	 30 		 05/02/2022 	 Biscuits
1012 Dark fantasy  	 33 	 8 		 01/10/2021 	 Biscuits
1013 Cadbury oreo  	 10 	 40 		 01/03/2022 	 Biscuits
1014 Hide & 